In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras
from keras.models import model_from_json
from skimage.transform import resize,rescale

Using TensorFlow backend.


In [41]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    
    ds = pydicom.dcmread(filename)  
    #print(ds)
    img = ds.pixel_array
    
    if (ds.Modality != 'DX'):
        img = None
    if(ds.BodyPartExamined != 'CHEST'):
        img = None
    if((ds.PatientPosition != 'PA') and (ds.PatientPosition != 'AP')):
        img = None
    if img is None:
        print('{} ... Failed Check'.format(filename))
        print('  ')
    else:
        print('Load file {} ...'.format(filename))
        print('Image type : {}, Body Part : {}, Patient Position : {}'.format(ds.Modality,ds.BodyPartExamined,ds.PatientPosition))
    
    
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    
    proc_img = img.copy()
    #img_mean = np.mean(proc_img)
    #img_std = np.std(proc_img)
    #proc_img = (proc_img - img_mean)/img_std
    proc_img = rescale(proc_img, 0.25, anti_aliasing=True)
    proc_img = resize(proc_img, (img_size),anti_aliasing=True)
    
    #plt.imshow(proc_img)

    
    
    
    #print(proc_img.shape[0])
     
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    json_file = open(model_path, 'r')
    loaded_model = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model)
    # load weights into new model
    model.load_weights(weight_path)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo
    #plt.imshow(img)
    #plt.xlabel("Prediction: " + ("PNEUMONIA" if model.predict(img) > thresh else "NORMAL"))
    prediction = ("PNEUMONIA" if model.predict(img) > thresh else "NO PNEUMONIA")
    #prediction = model.predict(img)
    
    #plt.show()
    
    return prediction 

In [3]:
my_model = load_model('model.json',"{}_my_model.best.hdf5".format('xray_class'))
my_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               12845568  
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)              

In [42]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'model.json'#path to saved model
weight_path = "{}_my_model.best.hdf5".format('xray_class')#path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0 
img_std = 0

my_model = load_model(model_path,weight_path)#loads model
thresh = 0.559925377368927#loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
    else:
        img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
        pred = predict_image(my_model,img_proc,thresh)
        
        print(pred)
        print('  ')


Load file test1.dcm ...
Image type : DX, Body Part : CHEST, Patient Position : PA
PNEUMONIA
  
Load file test2.dcm ...
Image type : DX, Body Part : CHEST, Patient Position : AP
PNEUMONIA
  
Load file test3.dcm ...
Image type : DX, Body Part : CHEST, Patient Position : AP
PNEUMONIA
  
test4.dcm ... Failed Check
  
test5.dcm ... Failed Check
  
test6.dcm ... Failed Check
  
